# Streaming with Converse Stream

We can stream data from Bedrock using the **converse_stream** function:

![Streaming text as it gets generated](https://everpath-course-content.s3-accelerate.amazonaws.com/instructor%2Fa46l9irobhg0f5webscixp0bs%2Fpublic%2F1748557673%2F05_-_008_-_Streaming_03.1748557673079.png)

## Steam Events

The stream event contains many event types within the response stream object:

![Stream event components](https://everpath-course-content.s3-accelerate.amazonaws.com/instructor%2Fa46l9irobhg0f5webscixp0bs%2Fpublic%2F1748557673%2F05_-_008_-_Streaming_12.1748557673618.png)

In [1]:
import boto3

In [4]:
client = boto3.client("bedrock-runtime", region_name="us-east-2")
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"


def add_user_message(messages, text):
    user_message = {"role": "user", "content": [{"text": text}]}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": [{"text": text}]}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {"temperature": temperature},
    }

    if system:
        params["system"] = [{"text": system}]

    response = client.converse(**params)

    return response["output"]["message"]["content"][0]["text"]

In [13]:
messages = []

add_user_message(messages, "Write a 1 sentence description of a fake database")
response = client.converse_stream(messages=messages, modelId=model_id)

print(response)

{'ResponseMetadata': {'RequestId': '4baa0075-f4ae-4eee-bb16-db1fe1c3f724', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 23 Aug 2025 15:33:59 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '4baa0075-f4ae-4eee-bb16-db1fe1c3f724'}, 'RetryAttempts': 0}, 'stream': <botocore.eventstream.EventStream object at 0x10b9a7b10>}


In [11]:
# printing the response stream object
print(response["stream"])

In [16]:
messages = []

add_user_message(messages, "Write a 1 sentence description of a fake database")
response = client.converse_stream(messages=messages, modelId=model_id)

for event in response["stream"]:
    print(event)

{'messageStart': {'role': 'assistant'}}
{'contentBlockDelta': {'delta': {'text': 'F'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': 'al'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': 'seData'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': ' is a fictional database'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': ' that pur'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': 'port'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': 'edly contains '}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': '10'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': ' '}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': 'billion'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': ' fabricated records of'}, 'contentBlockIndex': 0}}
{'contentBlockDelta': {'delta': {'text': ' non-existent individuals'}, 'contentBlockInd

For basic text generation, we just need the 'contentBlockDelta'. 

In [14]:
text = ""
for event in response["stream"]:
    if 'contentBlockDelta' in event:
        chunk = event["contentBlockDelta"]["delta"]["text"]
        print(chunk, end="") # end="" so that it doesn't print on the next line
        text += chunk

print("\n\nTotal Message: \n"+text)

A fictional database named "QuantumVault" allegedly contains every decision never made, storing parallel realities in quantum-encrypted fields that dynamically expand based on the observer's thought patterns.

Total Message: 
A fictional database named "QuantumVault" allegedly contains every decision never made, storing parallel realities in quantum-encrypted fields that dynamically expand based on the observer's thought patterns.


Note: Once I executed the for loop, I can't get to loop through the event anymore.